In [1]:
from datasets import load_dataset, concatenate_datasets
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)
import torch
import numpy as np


2026-01-23 07:01:24.647881: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769151684.841879      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769151684.897484      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769151685.356724      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769151685.356764      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769151685.356767      55 computation_placer.cc:177] computation placer alr

In [2]:
model_name = "facebook/mbart-large-50"
tokenizer = AutoTokenizer.from_pretrained(model_name, legacy=False)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

print(f"Model loaded: {model_name}")
print(f"Vocabulary size: {len(tokenizer)}")

tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

Model loaded: facebook/mbart-large-50
Vocabulary size: 250054


In [3]:
# Load datasets
ds_summary = load_dataset("sanjeev-bhandari01/nepali-summarization-dataset")
ds_translation = load_dataset("ashokpoudel/nepali-english-translation-dataset")

print(f"Summary dataset: {ds_summary}")
print(f"Translation dataset: {ds_translation}")

README.md:   0%|          | 0.00/263 [00:00<?, ?B/s]

Summerization_dataset.csv:   0%|          | 0.00/310M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/285558 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/368 [00:00<?, ?B/s]

data/train-00000-of-00009.parquet:   0%|          | 0.00/161M [00:00<?, ?B/s]

...

data/train-00006-of-00009.parquet:   0%|          | 0.00/270M [00:00<?, ?B/s]

data/train-00007-of-00009.parquet:   0%|          | 0.00/246M [00:00<?, ?B/s]

data/train-00008-of-00009.parquet:   0%|          | 0.00/246M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3560496 [00:00<?, ? examples/s]

Summary dataset: DatasetDict({
    train: Dataset({
        features: ['article', 'title', 'article_length'],
        num_rows: 285558
    })
})
Translation dataset: DatasetDict({
    train: Dataset({
        features: ['text', 'en', 'np'],
        num_rows: 3560496
    })
})


In [5]:
# Sample 10,000 examples from each dataset
ds_summary_sampled = ds_summary['train'].select(range(50000))
ds_translation_sampled = ds_translation['train'].select(range(50000))

print(f"Summary samples: {len(ds_summary_sampled)}")
print(f"Translation samples: {len(ds_translation_sampled)}")

Summary samples: 50000
Translation samples: 50000


In [6]:
def preprocess_summary_data(example):
    """Add task prefix for summarization"""
    return {
        'input_text': 'summarize: ' + example['article'],
        'target_text': example['title']
    }

def preprocess_translation_data(example):
    """Add task prefix for translation"""
    return {
        'input_text': 'translate English to Nepali: ' + example['en'],
        'target_text': example['np']
    }

# Apply preprocessing
ds_summary_prepared = ds_summary_sampled.map(preprocess_summary_data)
ds_translation_prepared = ds_translation_sampled.map(preprocess_translation_data)

print("Preprocessing complete!")
print(f"Sample input: {ds_summary_prepared[0]['input_text'][:100]}...")
print(f"Sample target: {ds_summary_prepared[0]['target_text']}")

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Preprocessing complete!
Sample input: summarize: उद्योग वाणिज्य तथा आपूर्ति मन्त्री मातृकाप्रसाद यादवले काम गर्न छोडेर राजनीति गर्ने कर्मच...
Sample target: राजनीति गर्ने कर्मचारीलाई नछाड्ने मन्त्री यादवको चेतावनी


In [7]:
# Combine both datasets
combined_dataset = concatenate_datasets([ds_summary_prepared, ds_translation_prepared])

# Split into train/eval
split_datasets = combined_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = split_datasets['train']
eval_dataset = split_datasets['test']

print(f"Combined dataset size: {len(combined_dataset)}")
print(f"Training samples: {len(train_dataset)}")
print(f"Evaluation samples: {len(eval_dataset)}")

Combined dataset size: 100000
Training samples: 80000
Evaluation samples: 20000


In [8]:
def tokenize_function(examples):
    """
    FIXED VERSION: Properly tokenizes inputs and labels using `text_target`.
    This should fix the zero loss issue!
    """
    # Tokenize inputs
    model_inputs = tokenizer(
        examples['input_text'],
        max_length=512,
        truncation=True,
        padding=False  # Data collator will handle padding
    )

    # Use text_target for label tokenization (recommended way)
    labels = tokenizer(
        text_target=examples['target_text'],
        max_length=128,
        truncation=True,
        padding=False
    )

    # Set labels
    model_inputs['labels'] = labels['input_ids']

    return model_inputs

In [9]:
tokenizer.tgt_lang = "ne_NP"

# Tokenize with batching for efficiency
tokenized_train = train_dataset.map(
    tokenize_function,
    batched=True,
    batch_size=1000,
    remove_columns=['article', 'title', 'article_length', 'text', 'en', 'np', 'input_text', 'target_text']
)

tokenized_eval = eval_dataset.map(
    tokenize_function,
    batched=True,
    batch_size=1000,
    remove_columns=['article', 'title', 'article_length', 'text', 'en', 'np', 'input_text', 'target_text']
)

print("Tokenization complete!")
print(f"Tokenized train dataset: {tokenized_train}")
print(f"Tokenized eval dataset: {tokenized_eval}")

Map:   0%|          | 0/80000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Tokenization complete!
Tokenized train dataset: Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 80000
})
Tokenized eval dataset: Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 20000
})


In [10]:
print("\n" + "="*50)
print("DIAGNOSTIC CHECK - Verifying Tokenization")
print("="*50)

sample = tokenized_train[0]

print(f"\n✓ Input IDs length: {len(sample['input_ids'])}")
print(f"✓ Labels length: {len(sample['labels'])}")
print(f"✓ First 10 input IDs: {sample['input_ids'][:10]}")
print(f"✓ First 10 labels: {sample['labels'][:10]}")

# Check for -100 (padding token in labels)
num_padding = sum(1 for x in sample['labels'] if x == -100)
print(f"✓ Number of -100 tokens in labels: {num_padding}")

# Decode to verify
valid_labels = [x for x in sample['labels'] if x != -100]
print(f"\n✓ Decoded input (first 100 chars):")
print(f"  {tokenizer.decode(sample['input_ids'][:50], skip_special_tokens=True)}")
print(f"\n✓ Decoded label (first 100 chars):")
print(f"  {tokenizer.decode(valid_labels[:50], skip_special_tokens=True)}")

print("\n" + "="*50)
print("If labels show valid text above, tokenization is correct!")
print("="*50 + "\n")


DIAGNOSTIC CHECK - Verifying Tokenization

✓ Input IDs length: 74
✓ Labels length: 9
✓ First 10 input IDs: [250004, 29334, 14096, 13, 12, 4702, 774, 94244, 36548, 115606]
✓ First 10 labels: [250018, 4702, 20, 94244, 115606, 71651, 3264, 6699, 2]
✓ Number of -100 tokens in labels: 0

✓ Decoded input (first 100 chars):
  summarize: नेपाल र जापानबीच हवाई सेवा सम्झौता हुने भएको छ । जापानका विदेशमन्त्री तारो कोनोको औपचारिक भ्रमणका क्रममा बुधबार दुई देशबीच त्यस्तो सम्झौता हुन लागेको हो । परराष्ट्र मन्त्री प्रदीपकुमार ज्ञवाली र उनका समकक्षी

✓ Decoded label (first 100 chars):
  नेपाल - जापान हवाई सम्झौता आज हुने

If labels show valid text above, tokenization is correct!



In [11]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    label_pad_token_id=-100,  # Explicitly set padding token for labels
    padding=True
)

print("Data collator created!")

Data collator created!


In [12]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,  # Effective batch size = 8
    warmup_steps=500,
    weight_decay=0.01,
    learning_rate=5e-5,

    # Logging
    logging_dir="./logs",
    logging_steps=50,
    logging_first_step=True,  # Log first step to verify loss > 0

    # Evaluation
    # The 'evaluation_strategy' argument is not recognized by the current transformers version.
    # It has been removed to fix the TypeError. Please update your transformers library.
    # Setting load_best_model_at_end to False as evaluation_strategy cannot be set.
    eval_steps=500,

    # Saving
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=False, # Changed to False to resolve ValueError

    # Generation (for evaluation)
    predict_with_generate=True,
    generation_max_length=128,

    # Performance
    fp16=False, # Changed to False to address nan loss

    # Disable wandb
    report_to='none',
)

print("Training arguments configured!")
print(f"Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")

Training arguments configured!
Effective batch size: 8


In [13]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

print("Trainer initialized successfully!")

/tmp/ipykernel_55/387213702.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Trainer initialized successfully!


In [14]:
print("\n" + "="*50)
print("TESTING ONE BATCH (Sanity Check)")
print("="*50)

# Get one batch
sample_batch = [tokenized_train[i] for i in range(4)]
batch = data_collator(sample_batch)

# Move to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
batch = {k: v.to(device) for k, v in batch.items()}

# Forward pass
model.eval()
with torch.no_grad():
    outputs = model(**batch)
    loss = outputs.loss

print(f"\n✓ Test batch loss: {loss.item():.4f}")

if loss.item() > 0:
    print("✅ LOSS IS POSITIVE - Ready to train!")
else:
    print("❌ LOSS IS ZERO - There's still an issue!")

print("="*50 + "\n")


TESTING ONE BATCH (Sanity Check)

✓ Test batch loss: 8.2703
✅ LOSS IS POSITIVE - Ready to train!



In [15]:
print("Starting training...")
print("Monitor the first few steps - loss should be > 0")
print("-" * 50)

trainer.train()

print("\n✅ Training complete!")

Starting training...
Monitor the first few steps - loss should be > 0
--------------------------------------------------


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


RuntimeError: [enforce fail at inline_container.cc:664] . unexpected pos 3759936384 vs 3759936272

In [16]:
print("Evaluating model...")

metrics = trainer.evaluate()

print("\n📊 Evaluation Metrics:")
for key, value in metrics.items():
    print(f"  {key}: {value}")

Evaluating model...

📊 Evaluation Metrics:
  eval_loss: 1.8901023864746094


In [17]:
# Save the fine-tuned model
output_dir = "./mt5-nepali-finetuned"
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"\n✅ Model saved to: {output_dir}")

OSError: [Errno 28] No space left on device: './mt5-nepali-finetuned'

In [1]:
def generate_summary(text):
    """Test the model on a sample"""
    input_text = f"summarize: {text}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    outputs = model.generate(
        **inputs,
        max_length=128,
        num_beams=4,
        early_stopping=True
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test with a sample from the dataset
test_article = train_dataset[0]['article']
print(f"Original article (first 200 chars):\n{test_article[:200]}...\n")

generated_summary = generate_summary(test_article)
actual_summary = train_dataset[0]['title']

print(f"Generated summary: {generated_summary}")
print(f"Actual summary: {actual_summary}")

NameError: name 'train_dataset' is not defined

In [2]:
from huggingface_hub import notebook_login

# Log in to Hugging Face Hub
notebook_login()


In [20]:

# Define your Hugging Face repository name
hf_repo_name = "mbart-large-50-nepali-finetuned-1"

# Push model and tokenizer to Hugging Face Hub
model.push_to_hub(hf_repo_name)
tokenizer.push_to_hub(hf_repo_name)

print(f"\n✅ Model and tokenizer pushed to Hugging Face Hub: {hf_repo_name}")

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            


✅ Model and tokenizer pushed to Hugging Face Hub: mbart-large-50-nepali-finetuned-1


# Task
Create functions for summarization and English to Nepali translation using the fine-tuned mBART-large-50 model, then build a Gradio interface to interact with these functions.

## Load Model and Tokenizer

### Subtask:
Load the fine-tuned mBART-large-50 model and tokenizer directly from the Hugging Face Hub repository.


In [21]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("sagunrai/mbart-large-50-nepali-finetuned-1")
model = AutoModelForSeq2SeqLM.from_pretrained("sagunrai/mbart-large-50-nepali-finetuned-1")

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/992 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

# 
Task
Load the fine-tuned mBART-large-50 model and tokenizer from "sagunrai/mbart-large-50-nepali-finetuned" from the Hugging Face Hub, create a Python function to summarize articles and another to translate English text to Nepali using the loaded model, and finally build an interactive Gradio interface to demonstrate these functionalities.

## Load Model and Tokenizer

### Subtask:
Ensure the fine-tuned mBART-large-50 model and tokenizer are loaded from 'sagunrai/mbart-large-50-nepali-finetuned' on the Hugging Face Hub and moved to the appropriate device (GPU/CPU).


**Reasoning**:
Load the fine-tuned model and tokenizer from Hugging Face Hub and move the model to the appropriate device (GPU or CPU) for efficient computation, as per the subtask instructions. This ensures the model is ready for inference.



In [22]:
import torch

hf_repo_name = "sagunrai/mbart-large-50-nepali-finetuned-1"

tokenizer = AutoTokenizer.from_pretrained(hf_repo_name)
model = AutoModelForSeq2SeqLM.from_pretrained(hf_repo_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(f"Model and tokenizer loaded from {hf_repo_name}")
print(f"Model moved to device: {device}")

Model and tokenizer loaded from sagunrai/mbart-large-50-nepali-finetuned-1
Model moved to device: cuda


## Create Summarization Function

### Subtask:
Develop a Python function that takes an article as input, applies the 'summarize:' prefix, tokenizes it, and generates a summary using the loaded model.


**Reasoning**:
Following the instructions, I will create a Python function `summarize_text` that takes an article, prepends the 'summarize:' prefix, tokenizes it, generates a summary using the loaded model, and then decodes and returns the result.



In [23]:
def summarize_text(text):
    """
    Generates a summary for the given text using the fine-tuned mBART-large-50 model.
    """
    input_text = f"summarize: {text}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Generate summary
    outputs = model.generate(
        **inputs,
        max_length=128,  # Max length of the generated summary
        num_beams=10,     # Number of beams for beam search
        early_stopping=True # Stop when all hypotheses have met the end of sequence criteria
    )

    # Decode the generated tokens to text
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

print("Summarization function 'summarize_text' created successfully.")

Summarization function 'summarize_text' created successfully.


## Create English to Nepali Translation Function

### Subtask:
Develop a Python function that takes an English text as input, applies the 'translate English to Nepali:' prefix, tokenizes it, and generates a Nepali translation using the loaded model.

## Create English to Nepali Translation Function

### Subtask:
Develop a Python function that takes an English text as input, applies the 'translate English to Nepali:' prefix, tokenizes it, and generates a Nepali translation using the loaded model.

#### Instructions
1. Define a function, for example, `translate_en_to_ne`, that accepts one argument: `text` (representing the English text to be translated).
2. Inside the function, prepend the string 'translate English to Nepali: ' to the input `text` to form the `input_text`.
3. Tokenize the `input_text` using the pre-loaded `tokenizer` (e.g., `tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)`).
4. Move the tokenized inputs to the appropriate `device` (GPU/CPU) using `.to(device)`.
5. Set the tokenizer's target language to Nepali (`tokenizer.src_lang = "en_XX"` and `tokenizer.tgt_lang = "ne_NP"`).
6. Generate the translation using the pre-loaded `model`'s `generate` method, passing the tokenized inputs, `max_length=128`, `num_beams=4`, and `early_stopping=True`.
7. Decode the generated output back into a human-readable string using the `tokenizer.decode()` method, ensuring `skip_special_tokens=True`.
8. Return the decoded translation.

#### Instructions
1. Define a function, for example, `translate_en_to_ne`, that accepts one argument: `text` (representing the English text to be translated).
2. Inside the function, prepend the string 'translate English to Nepali: ' to the input `text` to form the `input_text`.
3. Tokenize the `input_text` using the pre-loaded `tokenizer` (e.g., `tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)`).
4. Move the tokenized inputs to the appropriate `device` (GPU/CPU) using `.to(device)`.
5. Set the tokenizer's target language to Nepali (`tokenizer.src_lang = "en_XX"` and `tokenizer.tgt_lang = "ne_NP"`).
6. Generate the translation using the pre-loaded `model`'s `generate` method, passing the tokenized inputs, `max_length=128`, `num_beams=4`, and `early_stopping=True`.
7. Decode the generated output back into a human-readable string using the `tokenizer.decode()` method, ensuring `skip_special_tokens=True`.
8. Return the decoded translation.

#### Instructions
1. Define a function, for example, `translate_en_to_ne`, that accepts one argument: `text` (representing the English text to be translated).
2. Inside the function, prepend the string 'translate English to Nepali: ' to the input `text` to form the `input_text`.
3. Tokenize the `input_text` using the pre-loaded `tokenizer` (e.g., `tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)`).
4. Move the tokenized inputs to the appropriate `device` (GPU/CPU) using `.to(device)`.
5. Set the tokenizer's target language to Nepali (`tokenizer.src_lang = "en_XX"` and `tokenizer.tgt_lang = "ne_NP"`).
6. Generate the translation using the pre-loaded `model`'s `generate` method, passing the tokenized inputs, `max_length=128`, `num_beams=4`, and `early_stopping=True`.
7. Decode the generated output back into a human-readable string using the `tokenizer.decode()` method, ensuring `skip_special_tokens=True`.
8. Return the decoded translation.

**Reasoning**:
Following the detailed instructions, I will now implement the `translate_en_to_ne` Python function. This will involve defining the function, preparing the input text with the correct prefix, tokenizing it, setting the tokenizer's source and target languages, generating the translation using the model, and then decoding the output.



In [24]:
def translate_en_to_ne(text):
    """
    Translates English text to Nepali using the fine-tuned mBART-large-50 model.
    """
    input_text = f"translate English to Nepali: {text}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Set tokenizer languages for translation
    tokenizer.src_lang = "en_XX"
    tokenizer.tgt_lang = "ne_NP"

    # Generate translation
    outputs = model.generate(
        **inputs,
        max_length=128,  # Max length of the generated translation
        num_beams=4,     # Number of beams for beam search
        early_stopping=True # Stop when all hypotheses have met the end of sequence criteria
    )

    # Decode the generated tokens to text
    translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translation

print("Translation function 'translate_en_to_ne' created successfully.")

Translation function 'translate_en_to_ne' created successfully.


In [32]:
# --- TEST SAMPLES FOR SUMMARIZATION (NEPALI) ---
nepali_texts = [
    "नेपाल एक सुन्दर र विविध संस्कृति भएको देश हो। यहाँका हिमाल, पहाड र तराईका भूभागले गर्दा यो संसारभरि चिनिएको छ। पर्यटन नेपालको मुख्य आयस्रोतमध्ये एक हो। सगरमाथा र लुम्बिनी जस्ता पर्यटकीय स्थलहरूले गर्दा यहाँ धेरै विदेशी पर्यटकहरू आउने गर्छन्।",
    "कृत्रिम बुद्धिमत्ता (AI) ले अहिलेको आधुनिक युगमा ठूलो परिवर्तन ल्याइरहेको छ। यसले स्वास्थ्य, शिक्षा र प्रविधिको क्षेत्रमा नयाँ सम्भावनाहरू खोलेको छ। मानिसको कामलाई सजिलो बनाउन र समयको बचत गर्न AI धेरै सहयोगी सावित भएको छ।",
    "स्वस्थ जीवनशैलीका लागि सन्तुलित आहार र नियमित व्यायाम निकै महत्वपूर्ण छ। जंक फूडको बढ्दो प्रयोगले गर्दा मानिसहरूमा विभिन्न स्वास्थ्य समस्याहरू देखिन थालेका छन्। त्यसैले, हामीले आफ्नो खानपानमा ध्यान दिनुपर्छ र दैनिक शारीरिक क्रियाकलापमा सहभागी हुनुपर्छ।",
    "नेपाली साहित्यको इतिहास निकै पुरानो र धनी छ। भानुभक्त आचार्य, लक्ष्मीप्रसाद देवकोटा र लेखनाथ पौड्याल जस्ता दिग्गज साहित्यकारहरूले नेपाली भाषालाई उचाइमा पुर्याउनु भएको छ। आजको समयमा पनि नयाँ पुस्ताले यसलाई निरन्तरता दिइरहेका छन्।",
    "जलवायु परिवर्तन आजको विश्वको सबैभन्दा ठूलो चुनौती बनेको छ। बढ्दो तापमानका कारण हिमालका हिउँहरू पग्लिरहेका छन् र समुद्रको सतह बढ्दै गइरहेको छ। यसलाई रोक्नका लागि सबै देशहरू मिलेर प्रदूषण कम गर्न र वन संरक्षणमा जोड दिनुपर्छ।"
]

# --- TEST SAMPLES FOR TRANSLATION (ENGLISH TO NEPALI) ---
english_texts = [
    "Hello, how are you doing today? I hope you have a wonderful morning.",
    "The capital city of Nepal is Kathmandu, which is famous for its ancient temples and culture.",
    "Technology is advancing very fast, and it is important for students to learn coding and digital skills.",
    "Health is wealth, so we should always try to eat organic food and drink plenty of water.",
    "I am learning how to use machine learning models to solve real-world problems in my country."
]

# --- TESTING THE FUNCTIONS ---

print("\n--- Testing Summarization (Nepali) ---")
for i, text in enumerate(nepali_texts):
    summary = summarize_text(text)
    print(f"\nOriginal {i+1}: {text}")
    print(f"Summary {i+1}: {summary}")

print("\n" + "="*50)

print("\n--- Testing Translation (English to Nepali) ---")
for i, text in enumerate(english_texts):
    translation = translate_en_to_ne(text)
    print(f"\nEnglish {i+1}: {text}")
    print(f"Nepali {i+1}: {translation}")


--- Testing Summarization (Nepali) ---

Original 1: नेपाल एक सुन्दर र विविध संस्कृति भएको देश हो। यहाँका हिमाल, पहाड र तराईका भूभागले गर्दा यो संसारभरि चिनिएको छ। पर्यटन नेपालको मुख्य आयस्रोतमध्ये एक हो। सगरमाथा र लुम्बिनी जस्ता पर्यटकीय स्थलहरूले गर्दा यहाँ धेरै विदेशी पर्यटकहरू आउने गर्छन्।
Summary 1: सगरमाथा र लुम्बिनीमा विदेशी पर्यटकको घुइँचो

Original 2: कृत्रिम बुद्धिमत्ता (AI) ले अहिलेको आधुनिक युगमा ठूलो परिवर्तन ल्याइरहेको छ। यसले स्वास्थ्य, शिक्षा र प्रविधिको क्षेत्रमा नयाँ सम्भावनाहरू खोलेको छ। मानिसको कामलाई सजिलो बनाउन र समयको बचत गर्न AI धेरै सहयोगी सावित भएको छ।
Summary 2: कृत्रिम बुद्धिमत्ताले ल्याएको परिवर्तन

Original 3: स्वस्थ जीवनशैलीका लागि सन्तुलित आहार र नियमित व्यायाम निकै महत्वपूर्ण छ। जंक फूडको बढ्दो प्रयोगले गर्दा मानिसहरूमा विभिन्न स्वास्थ्य समस्याहरू देखिन थालेका छन्। त्यसैले, हामीले आफ्नो खानपानमा ध्यान दिनुपर्छ र दैनिक शारीरिक क्रियाकलापमा सहभागी हुनुपर्छ।
Summary 3: जंक फूडबाट कसरी बच्न सकिन्छ ?

Original 4: नेपाली साहित्यको इतिहास निकै पुरानो र धनी छ। 

## Build Gradio Interface

### Subtask:
Create a Gradio interface that allows users to switch between summarization and English to Nepali translation, using the functions developed in the previous steps.

## Build Gradio Interface

### Subtask:
Create a Gradio interface that allows users to switch between summarization and English to Nepali translation, using the functions developed in the previous steps.

**Reasoning**:
The subtask now requires building the Gradio interface. I will first import the necessary `gradio` library.



In [25]:
import gradio as gr
print("Gradio library imported successfully.")

Gradio library imported successfully.


**Reasoning**:
Now that Gradio is imported and the summarization and translation functions are ready, I will define and launch the Gradio interface to allow users to interact with these functions. This will complete the current subtask of building the Gradio interface.



In [31]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("# 🇳🇵 Nepali Multitask Model (Summarization & Translation)")

    with gr.Tab("Summarization"):
        summary_input = gr.Textbox(
            lines=8,
            label="Enter text to summarize (Nepali)",
            value="नेपाल दक्षिण एसियामा अवस्थित एक सुन्दर हिमाली देश हो। यहाँ हिमाल, पहाड र तराई क्षेत्र रहेका छन्।"
        )
        summary_output = gr.Textbox(
            label="Generated Summary (Nepali)",
            interactive=False
        )
        summary_button = gr.Button("Summarize")

        summary_button.click(
            fn=summarize_text,
            inputs=summary_input,
            outputs=summary_output
        )

    with gr.Tab("English → Nepali Translation"):
        translation_input = gr.Textbox(
            lines=8,
            label="Enter English text to translate",
            value="Nepal is a beautiful country with diverse geography."
        )
        translation_output = gr.Textbox(
            label="Translated Text (Nepali)",
            interactive=False
        )
        translation_button = gr.Button("Translate")

        translation_button.click(
            fn=translate_en_to_ne,
            inputs=translation_input,
            outputs=translation_output
        )

demo.launch(share=True, debug=True)

print("✅ Gradio interface launched successfully.")


* Running on local URL:  http://127.0.0.1:7865

Could not create share link. Missing file: /root/.cache/huggingface/gradio/frpc/frpc_linux_amd64_v0.3. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.3/frpc_linux_amd64
2. Rename the downloaded file to: frpc_linux_amd64_v0.3
3. Move the file to this location: /root/.cache/huggingface/gradio/frpc


Keyboard interruption in main thread... closing server.
✅ Gradio interface launched successfully.
